In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch_geometric
from torch_geometric.loader import DataLoader
import yaml

import Dataset
import Models
from utils import time_func

In [2]:
print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Cuda device: {torch.cuda.get_device_name()}")
print(f"Torch geometric version: {torch_geometric.__version__}")

Torch version: 2.0.1+cu117
Cuda available: False
Torch geometric version: 2.3.1


In [3]:
yaml_file = './config_pipeline.yaml'
params = yaml.safe_load(open(yaml_file))

data_path = params['input_subset_pre_processed']

In [4]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')